pip install docplex gurobipy

2. Preparación del modelo base
El siguiente es un ejemplo genérico para un modelo F-MDRP basado en las restricciones SCF (Single Commodity Flow):

In [8]:
#Librerias necesarias
from docplex.mp.model import Model
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from pathlib import Path
import os
import time
import csv
from datetime import datetime

In [9]:
#Funcion Para Parsear los Archivos .dat
def read_instance(file_path):
    """
    Lee una instancia de archivo .dat y devuelve los parámetros del problema.
    Formato esperado:
    - Primera línea: Número de clientes
    - Segunda línea: Número de depósitos disponibles
    - Siguientes líneas: Coordenadas de los depósitos (x, y)
    - Luego: Coordenadas de los clientes (x, y)
    - Capacidades de los vehículos
    - Capacidades de los depósitos
    - Demandas de los clientes
    - Costos de apertura de los depósitos
    - Costos de apertura de una ruta (costo de un vehículo)
    - Indicación de si los costos son enteros (0) o reales (1)
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Leer el número de clientes y depósitos
    num_customers = int(lines[0].strip())
    num_depots = int(lines[1].strip())

    # Leer las coordenadas de los depósitos
    depots_coords = []
    line_index = 3
    for i in range(num_depots):
        line = lines[line_index].strip()
        x, y = map(float, line.replace('\t', ' ').split())
        depots_coords.append((x, y))
        line_index += 1
    line_index += 1

    # Leer las coordenadas de los clientes
    customers_coords = []
    for i in range(num_customers):
        line = lines[line_index].strip()
        x, y = map(float, line.replace('\t', ' ').split())
        customers_coords.append((x, y))
        line_index += 1
    line_index += 1

    # Leer la capacidad de los vehículos
    vehicle_capacity = float(lines[line_index].strip())
    line_index += 2

    # Leer las capacidades de los depósitos
    depots_capacities = []
    for i in range(num_depots):
        depots_capacities.append(float(lines[line_index].strip()))
        line_index += 1
    line_index += 1

    # Leer las demandas de los clientes
    customers_demands = []
    for i in range(num_customers):
        customers_demands.append(float(lines[line_index].strip()))
        line_index += 1
    line_index += 1

    # Leer los costos de apertura de los depósitos
    opening_costs = []
    for i in range(num_depots):
        opening_costs.append(float(lines[line_index].strip()))
        line_index += 1
    line_index += 1

    # Leer el costo de apertura de una ruta (costo de un vehículo)
    vehicle_route_cost = float(lines[line_index].strip())
    line_index += 2

    # Leer si los costos son enteros (0) o reales (1)
    cost_type = int(lines[line_index].strip())

    from_depots_arcs = [(i, j) for i in range(num_depots) for j in range(num_depots, num_depots+num_customers)]
    to_depots_arcs = [(j, i) for i in range(num_depots) for j in range(num_depots, num_depots+num_customers)]
    clients_arcs = [(i, j) for i in range(num_depots, num_depots+num_customers) for j in range(num_depots, num_depots+num_customers) if i != j]
    arcs = from_depots_arcs + to_depots_arcs + clients_arcs

    # Generar la matriz de costos entre depósitos y clientes (calculada usando la distancia euclidiana)
    costs = {}
    for i, j in arcs:
        if i < num_depots:
            c1_x = depots_coords[i][0]
            c1_y = depots_coords[i][1]
        else:
            c1_x = customers_coords[i-num_depots][0]
            c1_y = customers_coords[i-num_depots][1]
        if j < num_depots:
            c2_x = depots_coords[j][0]
            c2_y = depots_coords[j][1]
        else:
            c2_x = customers_coords[j-num_depots][0]
            c2_y = customers_coords[j-num_depots][1]
        costs[i, j] = np.sqrt((c1_x - c2_x)**2 + (c1_y - c2_y)**2)

    # Empaquetar todos los parámetros en un diccionario
    data = {
        'num_customers': num_customers,
        'num_depots': num_depots,
        'depots_coords': depots_coords,
        'customers_coords': customers_coords,
        'vehicle_capacity': vehicle_capacity,
        'depots_capacities': depots_capacities,
        'customers_demands': customers_demands,
        'opening_costs': opening_costs,
        'vehicle_route_cost': vehicle_route_cost,
        'cost_type': cost_type,
        'costs': costs,
        'from_depots_arcs' : from_depots_arcs,
        'to_depots_arcs' : to_depots_arcs,
        'clients_arcs' : clients_arcs,
        'arcs' : arcs
    }

    return data

In [ ]:
# Implementación del modelo SCF con CPLEX
def setup_model_scf_cplex(depots, clients, costs, vehicles_per_depot,from_depots_arcs, clients_arcs, arcs):
    mdl = Model(name="F-MDRP SCF")

    # Variables de decisión
    x = mdl.binary_var_dict(arcs,name="x")
    f = mdl.continuous_var_dict(from_depots_arcs+clients_arcs,name="f")

    # Función objetivo: Minimizar los costos de ruta
    mdl.minimize(mdl.sum(costs[i, j] * x[i, j] for i,j in arcs))

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.add_constraint(mdl.sum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.add_constraint(mdl.sum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.add_constraint(mdl.sum(x[d, j] for j in clients) == 1)
        mdl.add_constraint(mdl.sum(x[j, d] for j in clients) == 1)

    # 3. Eliminación de subrutas (SCF)
    for c in clients:
        mdl.add_constraint(mdl.sum(f[i, c] for i in depots + clients if i != c) -
                           mdl.sum(f[c, j] for j in clients if c != j) == 1)

    for i, j in [(i, j) for i in depots + clients for j in clients if i != j]:
        mdl.add_constraint(f[i, j] <= len(clients) * x[i, j])

    return mdl

In [10]:
# Implementación del modelo SCF con Gurobi
def setup_model_scf_gurobi(depots, clients, costs, vehicles_per_depot,from_depots_arcs, clients_arcs, arcs):
    mdl = gp.Model("F-MDRP SCF")

    # Variables de decisión
    x = mdl.addVars(arcs, vtype=GRB.BINARY, name="x")
    f = mdl.addVars(from_depots_arcs+clients_arcs, vtype=GRB.CONTINUOUS, name="f")

    # Función objetivo
    mdl.setObjective(
        gp.quicksum(costs[i, j] * x[i, j] for i,j in arcs),
        GRB.MINIMIZE
    )

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.addConstr(gp.quicksum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.addConstr(gp.quicksum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.addConstr(gp.quicksum(x[d, j] for j in clients) == 1)
        mdl.addConstr(gp.quicksum(x[j, d] for j in clients) == 1)

    # 3. Eliminación de subrutas (SCF)
    for c in clients:
        mdl.addConstr(gp.quicksum(f[i, c] for i in depots + clients if i != c) -
                      gp.quicksum(f[c, j] for j in clients if c != j) == 1)

    for i, j in [(i, j) for i in depots + clients for j in clients if i != j]:
        mdl.addConstr(f[i, j] <= len(clients) * x[i, j])

    # Solución
    mdl.setParam("TimeLimit", 3600)  # Límite de tiempo de 1 hora
    mdl.optimize()

    return mdl

In [ ]:
# Implementación del modelo NODE-DL con CPLEX
def setup_model_node_dl_cplex(depots, clients, costs, vehicles_per_depot, arcs):
    mdl = Model(name="F-MDRP SCF")

    # Variables de decisión
    x = mdl.binary_var_dict(arcs,name="x")
    k = mdl.continuous_var_dict(clients, lb=0, ub=len(clients), name="k")

    # Función objetivo: Minimizar los costos de ruta
    mdl.minimize(mdl.sum(costs[i,j] * x[i, j] for i,j in arcs))

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.add_constraint(mdl.sum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.add_constraint(mdl.sum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.add_constraint(mdl.sum(x[d, j] for j in clients) == 1)
        mdl.add_constraint(mdl.sum(x[j, d] for j in clients) == 1)

    # 3. Eliminación de rutas (NODE-DL)
    for i, j in [(i, j) for i in depots for j in clients if i != j]:
        mdl.add_constraint(k[j] + (len(depots)-1-i) * x[i,j] <= len(depots))
        mdl.add_constraint(k[j] + (len(depots)-1-i) * x[j,i] <= len(depots))
        mdl.add_constraint(1 + (i) * x[j,i] <= k[j])
        mdl.add_constraint(1 + (i) * x[i,j] <= k[j])
    for i in clients:
        for j in clients:
            if i != j:
                mdl.add_constraint(k[i] - k[j] <= (len(depots) - 1) * (1 - x[j,i] - x[i,j]))

    return mdl

In [11]:
# Implementación del modelo NODE-DL con Gurobi
def setup_model_node_dl_gurobi(depots, clients, costs, vehicles_per_depot, arcs):
    mdl = gp.Model("F-MDRP SCF")

    # Variables de decisión
    x = mdl.addVars(arcs, vtype=GRB.BINARY, name="x")
    k = mdl.addVars(clients, lb=0, ub=len(clients), vtype=GRB.CONTINUOUS, name="k")

    # Función objetivo
    mdl.setObjective(
        gp.quicksum(costs[i, j] * x[i, j] for i,j in arcs),
        GRB.MINIMIZE
    )

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.addConstr(gp.quicksum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.addConstr(gp.quicksum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.addConstr(gp.quicksum(x[d, j] for j in clients) == 1)
        mdl.addConstr(gp.quicksum(x[j, d] for j in clients) == 1)

    # 3. Eliminación de rutas (NODE-DL)  
    for i, j in [(i, j) for i in depots for j in clients if i != j]:
        mdl.addConstr(k[j] + (len(depots)-1-i) * x[i,j] <= len(depots))
        mdl.addConstr(k[j] + (len(depots)-1-i) * x[j,i] <= len(depots))
        mdl.addConstr(1 + (i) * x[j,i] <= k[j])
        mdl.addConstr(1 + (i) * x[i,j] <= k[j])
    for i in clients:
        for j in clients:
            if i != j:
                mdl.addConstr(k[i] - k[j] <= (len(depots) - 1) * (1 - x[j,i] - x[i,j]))

    # Solución
    mdl.setParam("TimeLimit", 3600)  # Límite de tiempo de 1 hora
    mdl.optimize()

    return mdl

In [ ]:
#Main para analizar una instancia
#region
CURRENT_DIR = Path(os.path.abspath('')).resolve()
instance_path = CURRENT_DIR / "instances" / "coord50-5-1.dat" 

# Leer la instancia
data = read_instance(instance_path)

# Datos extraídos
depots = list(range(data['num_depots']))
clients = list(range(data['num_depots'], data['num_depots'] + data['num_customers']))
costs = data['costs']
vehicles_per_depot = {d: 3 for d in depots}  # Número arbitrario de vehículos por depósito
from_depots_arcs = [(i, j) for i in depots for j in clients]
to_depots_arcs = [(j, i) for i in depots for j in clients]
clients_arcs = [(i, j) for i in clients for j in clients if i != j]
arcs = from_depots_arcs + to_depots_arcs + clients_arcs
#endregion

In [ ]:
# Testeo De Modelo SCF Usando Cplex Para una instancia
#region
# Definir tiempo de inicio Cplex
start_time_Cplex = time.time()

# Resolver con Modelo SCF en CPLEX
cplex_model = setup_model_scf_cplex(depots, clients, costs, vehicles_per_depot)
cplex_model.parameters.timelimit = 3600  # Límite de tiempo de 1 hora
cplex_model.solve()

# Tiempo de Finalizacion de Cplex
end_time_Cplex = time.time()

print("Resultados CPLEX:")
print(cplex_model.objective_value)
print("Tiempo: %s" % (end_time_Cplex-start_time_Cplex))
#endregion

In [ ]:
# Testeo De Modelo NODE-DL Usando Cplex Para una instancia
#region
# Definir tiempo de inicio Cplex
start_time_Cplex = time.time()

# Resolver con Modelo NODE-DL en CPLEX
cplex_model = setup_model_node_dl_cplex(depots, clients, costs, vehicles_per_depot, arcs)
cplex_model.parameters.timelimit = 3600  # Límite de tiempo de 1 hora
cplex_model.solve()

# Tiempo de Finalizacion de Cplex
end_time_Cplex = time.time()

print("Resultados CPLEX:")
print(cplex_model.objective_value)
print("Tiempo: %s" % (end_time_Cplex-start_time_Cplex))
#endregion

In [2]:
# Testeo De Modelo SCF Usando Gurobi Para una instancia
#region
# Definir tiempo de inicio Gurobi
start_time_Gurobi = time.time()

# Resolver con Modelo SCF en Gurobi
gurobi_model = setup_model_scf_gurobi(depots, clients, costs, vehicles_per_depot, from_depots_arcs, clients_arcs, arcs)

# Tiempo de Finalizacion de Gurobi
end_time_Gurobi = time.time()

print("Resultados Gurobi:")
if gurobi_model.status == GRB.OPTIMAL:
    print(gurobi_model.objVal)
print("Tiempo: %s" % (end_time_Gurobi-start_time_Gurobi))
#endregion

NameError: name 'setup_model_scf_gurobi' is not defined

In [19]:
# Testeo De Modelo NODE-DL Usando Gurobi Para una instancia
#region
# Definir tiempo de inicio Gurobi
start_time_Gurobi = time.time()

# Resolver con Modelo NODE-DL en Gurobi
gurobi_model = setup_model_node_dl_gurobi(depots, clients, costs, vehicles_per_depot, arcs)

# Tiempo de Finalizacion de Gurobi
end_time_Gurobi = time.time()

print("Resultados Gurobi:")
if gurobi_model.status == GRB.OPTIMAL:
    print(gurobi_model.objVal)
print("Tiempo: %s" % (end_time_Gurobi-start_time_Gurobi))
#endregion

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 2566890 - for non-commercial use only - registered to lu___@udec.cl
Optimize a model with 3560 rows, 3000 columns and 17500 nonzeros
Model fingerprint: 0x53c20475
Variable types: 50 continuous, 2950 integer (2950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 5e+01]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 1400.4047887
Presolve removed 800 rows and 0 columns
Presolve time: 0.03s
Presolved: 2760 rows, 3000 columns, 16700 nonzeros
Variable types: 50 continuous, 2950 integer (2950 binary)

Root relaxation: objective 2.799809e+02, 252 iterations, 0.01 seconds (0.01 work units)

    Nodes 

In [37]:
#Main para analziar multiples instancias
#region
CURRENT_DIR = Path(os.path.abspath('')).resolve()
instances_list = list(CURRENT_DIR.glob('instances/Instances_Barreto_LRP/*.dat')) #instances/Instances_Barreto_LRP/*.dat #instances/Instances_Prodhon_LRP/*.dat #instances/Instances_Tuzun_LRP/*.dat
depots_list = []
clients_list = []
costs_list = []
vehicles_per_depot_list = []
from_depots_arcs_list = []
to_depots_arcs_list = []
clients_arcs_list = []
arcs_list = []

#Nombre de Archivo CSV
base_filename = 'reports/report.csv'
timestamp = datetime.now().strftime('%Y%m%d%H%M')
new_filename = f"{base_filename.split('.')[0]}_{timestamp}.{base_filename.split('.')[-1]}"
with open(new_filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Instancia", "Modelo", "Solver", "Resultado", "Tiempo"])

print(instances_list)
# Leer la instancias
for i in range(len(instances_list)):
    data=read_instance(instances_list[i])
    # Datos extraídos
    depots_list.append(list(range(data['num_depots'])))
    clients_list.append(list(range(data['num_depots'], data['num_depots'] + data['num_customers'])))
    costs_list.append(data['costs'])
    vehicles_per_depot_list.append({d: 3 for d in depots_list[i]})  # Número arbitrario de vehículos por depósito
    from_depots_arcs_list.append([(j, k) for j in depots_list[i] for k in clients_list[i]])
    to_depots_arcs_list.append([(k, j) for j in depots_list[i] for k in clients_list[i]])
    clients_arcs_list.append([(j, k) for j in clients_list[i] for k in clients_list[i] if j != k])
    arcs_list.append(from_depots_arcs_list[i] + to_depots_arcs_list[i] + clients_arcs_list[i])
    print(depots_list[i])
#endregion

[WindowsPath('S:/Universidad/Optimizacion/Gurobi_Cplex_comparation/instances/Instances_Barreto_LRP/coordChrist100.dat'), WindowsPath('S:/Universidad/Optimizacion/Gurobi_Cplex_comparation/instances/Instances_Barreto_LRP/coordChrist50.dat'), WindowsPath('S:/Universidad/Optimizacion/Gurobi_Cplex_comparation/instances/Instances_Barreto_LRP/coordChrist75.dat')]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# Testeo De Modelo SCF Usando Cplex Para multiples Instancias
for i in range(len(instances_list)):
    # Definir tiempo de inicio Cplex
    start_time_Cplex = time.time()

    # Resolver con Modelo SCF en CPLEX
    cplex_model = setup_model_scf_cplex(depots_list[i], clients_list[i], costs_list[i], vehicles_per_depot_list[i], from_depots_arcs_list[i], clients_arcs_list[i], arcs_list[i])
    cplex_model.parameters.timelimit = 3600  # Límite de tiempo de 1 hora
    cplex_model.solve()

    # Tiempo de Finalizacion de Cplex
    end_time_Cplex = time.time()
    
    # Registro en CSV
    with open(new_filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([instances_list[i], "SCF", "Cplex", cplex_model.objective_value, end_time_Cplex-start_time_Cplex])

In [ ]:
# Testeo De Modelo NODE-DL Usando Cplex Para multiples Instancias
for i in range(len(instances_list)):
    # Definir tiempo de inicio Cplex
    start_time_Cplex = time.time()

    # Resolver con Modelo NODE-DL en CPLEX
    cplex_model = setup_model_node_dl_cplex(depots_list[i], clients_list[i], costs_list[i], vehicles_per_depot_list[i], arcs_list[i])
    cplex_model.parameters.timelimit = 3600  # Límite de tiempo de 1 hora
    cplex_model.solve()

    # Tiempo de Finalizacion de Cplex
    end_time_Cplex = time.time()

    # Registro en CSV
    with open(new_filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([instances_list[i], "NODE-DL", "Cplex", cplex_model.objective_value, end_time_Cplex-start_time_Cplex])

In [38]:
# Testeo De Modelo SCF Usando Gurobi Para multiples Instancias
for i in range(len(instances_list)):
    # Definir tiempo de inicio Gurobi
    start_time_Gurobi = time.time()

    # Resolver con Modelo SCF en Gurobi
    gurobi_model = setup_model_scf_gurobi(depots_list[i], clients_list[i], costs_list[i], vehicles_per_depot_list[i], from_depots_arcs_list[i], clients_arcs_list[i], arcs_list[i])

    # Tiempo de Finalizacion de Gurobi
    end_time_Gurobi = time.time()

    # Registro en CSV
    with open(new_filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([instances_list[i], "SCF", "Gurobi", gurobi_model.objVal, end_time_Gurobi-start_time_Gurobi])

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 2566890 - for non-commercial use only - registered to lu___@udec.cl
Optimize a model with 11220 rows, 22800 columns and 66400 nonzeros
Model fingerprint: 0x017ea25a
Variable types: 10900 continuous, 11900 integer (11900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.13s
Presolved: 11220 rows, 22800 columns, 66400 nonzeros
Variable types: 10900 continuous, 11900 integer (11900 binary)

Root relaxation: objective 6.024054e+02, 24399 iterations, 3.66 seconds (4.27 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

In [39]:
# Testeo De Modelo NODE-DL Usando Gurobi Para multiples Instancias
for i in range(len(instances_list)):
    # Definir tiempo de inicio Gurobi
    start_time_Gurobi = time.time()

    # Resolver con Modelo SCF en Gurobi
    gurobi_model = setup_model_node_dl_gurobi(depots_list[i], clients_list[i], costs_list[i], vehicles_per_depot_list[i], arcs_list[i])

    # Tiempo de Finalizacion de Gurobi
    end_time_Gurobi = time.time()

    # Registro en CSV
    with open(new_filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([instances_list[i], "NODE-DL", "Gurobi", gurobi_model.objVal, end_time_Gurobi-start_time_Gurobi])

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 2566890 - for non-commercial use only - registered to lu___@udec.cl
Optimize a model with 14120 rows, 12000 columns and 71000 nonzeros
Model fingerprint: 0xfa7fcaf2
Variable types: 100 continuous, 11900 integer (11900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 3669.9394379
Presolve removed 3600 rows and 0 columns
Presolve time: 0.24s
Presolved: 10520 rows, 12000 columns, 67400 nonzeros
Variable types: 100 continuous, 11900 integer (11900 binary)

Root relaxation: objective 6.371921e+02, 659 iterations, 0.05 seconds (0.05 work units)
